# Notebook 3: Retrieval + Augmented + Generation (RAG)

En este notebook unificamos las tres partes del sistema RAG que hemos visto:

1. **Retrieval**: buscar información relevante de una base de datos vectorial
2. **Augmented**: construir el *contexto* textual a partir de los documentos recuperados
3. **Generation**: generar una respuesta usando un modelo de lenguaje (LLM)

## 1. Configurar conexión a Weaviate

In [1]:
import weaviate
from sentence_transformers import SentenceTransformer
import requests

client = weaviate.connect_to_local(
    host="rag-weaviate",
    port=8080,
    grpc_port=50051
)
collection = client.collections.get("Note")

## 2. Configurar conexión a Ollama

In [4]:
import requests

OLLAMA_URL = "http://rag-ollama:11434"
MODEL = "gemma2:2b" 

def ask_ollama(prompt: str):
    response = requests.post(
        f"{OLLAMA_URL}/api/generate",
        json={
            "model": MODEL,
            "prompt": prompt,
            "stream": False
        }
    )
    print(response.json()["response"])

ask_ollama("Hola, ¿Cómo estas?")

¡Hola! 😊 Estoy muy bien, gracias por preguntar.  ¿Y tú, cómo estás? 



## 3. Cargar modelo de embeddings

In [5]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")

## 4. Definir consulta, vectorizar y recuperar documentos

In [11]:
query = "¿Qué ventajas tiene usar procesamiento distribuido con Hadoop?"

vector = embedder.encode(query)

results = collection.query.hybrid(
    query=query,
    vector=vector,
    alpha=0.5,
    limit=3,
    include_vector=False,
    return_metadata=["score"]
)

for r in results.objects:
    print(f"[{r.metadata.score:.3f}] {r.properties['title']} - {r.properties['subject']}")

[1.000] Componentes de Hadoop - Infraestructuras de Big Data
[0.479] Control de versiones con Git - Fundamentos de Ingeniería del Software
[0.079] Ciclo de vida del software - Fundamentos de Ingeniería del Software


## 5. Construir contexto a partir de las notas recuperadas

In [12]:
chunks = []
for r in results.objects:
    title = r.properties["title"]
    subject = r.properties["subject"]
    year = r.properties["year"]
    text = r.properties["text"]
    chunks.append(f"[{title} ({subject}, {year})]: {text}")

context = "\n".join(chunks)

print("Contexto generado para el modelo:\n")
print(context)


Contexto generado para el modelo:

[Componentes de Hadoop (Infraestructuras de Big Data, 2024.0)]: Hadoop está compuesto por HDFS para almacenamiento, YARN para gestión de recursos y MapReduce para procesamiento.
[Control de versiones con Git (Fundamentos de Ingeniería del Software, 2023.0)]: Git permite gestionar versiones del código fuente de forma distribuida, facilitando el trabajo colaborativo entre desarrolladores.
[Ciclo de vida del software (Fundamentos de Ingeniería del Software, 2023.0)]: El ciclo de vida del software incluye etapas como análisis de requisitos, diseño, implementación, pruebas y mantenimiento.


## 6. Construir prompt final y generar respuesta

In [13]:
prompt = (
    f"A continuación tienes apuntes de clase relevantes:\n\n"
    f"{context}\n\n"
    f"En base a estos apuntes, responde a esta pregunta:\n{query}"
)

ask_ollama(prompt)

El procesamiento distribuido con Hadoop ofrece varias ventajas clave: 

**1. Escalabilidad:**  Puedes agregar más nodos al cluster de Hadoop cuando necesites procesar más datos. Esto permite que el sistema se escalase para manejar grandes conjuntos de datos sin problemas.

**2. Eficiencia en el almacenamiento y procesamiento:** HDFS (Hadoop Distributed File System) organiza los datos de forma eficiente, optimizando el acceso y la velocidad de procesamiento. 

**3.  Flexibilidad:**  MapReduce, un framework de código abierto para procesamiento distribuido, permite a los usuarios ejecutar tareas de análisis de datos en diferentes nodos del cluster, lo que les da gran flexibilidad.

**4.  Reducción de tiempo de procesamiento:** Al dividir las tareas de procesamiento en múltiples nodos, Hadoop reduce el tiempo de ejecución del trabajo y facilita la optimización de recursos. 

**5.  Mejor control de errores:** La naturaleza distribuida de Hadoop permite detectar y solucionar errores de forma